# Lecture 7: Option Pricing

In [1]:
import numpy as np
import numpy.random as npr

## Binomial tree

In [2]:
def binomialtree(T,N,S,r,sigma,K, typeEA, typeCP):

    if typeCP == "call": # Call or put option
        typeCP = 1
    else:
        typeCP = -1

    dt = np.float(T)/N # step size
    u = np.exp(sigma*np.sqrt(dt)) # up and down steps
    d = 1/u
    p = (np.exp(r*dt)-d)/(u-d) # risk-neutral probabilities

    ST = np.zeros(N+1) # Final stock prices vector
    option = np.zeros(N+1) # option price vector

    for i in range(N+1):
        ST[i] = S*u**(N-i)*d**i # Fill in terminal stock prices
        option[i] = max(typeCP*(ST[i]-K),0) # Fill in terminal option prices
    
    for i in range(N-1,-1,-1):
        for j in range(i+1):
            option[j] = np.exp(-r*dt)*(p*option[j]+(1-p)*option[j+1])
            if typeEA == "American":
                ST[j] = np.exp(-r*dt)*(p*ST[j]+(1-p)*ST[j+1])
                option[j] = max(option[j], max(typeCP*(ST[j]-K),0))

    return option[0]

In [35]:
binomialtree(2, 500, 100, 0.05, 0.10, 100, "Euro", "put")

1.8932270788587007

In [34]:
binomialtree(2, 500, 100, 0.05, 0.10, 100, "American", "put")

2.8754584823327676

## Monte Carlo simulation (static, plain vanilla call & put)

In [56]:
def montecarlo(T,N,S,r,sigma,K,typeCP):

    typeCP = 1 if typeCP == "call" else -1
    z = npr.randn(N)
    # simulate index level at maturity
    ST = S*np.exp((r-0.5*sigma**2)*T + sigma*np.sqrt(T)*z)
    # calculate payoff at maturity
    option = np.maximum(typeCP*(ST - K), 0)
    option = np.exp(-r*T)*option.mean()
    return option

In [58]:
montecarlo(2, 10**7, 100, 0.05, 0.10, 100, "put")

1.8983150606145902

## Monte Carlo simulation (dynamic, i.e., Asian options)

In [74]:
def montecarlo_dynamic(T,N,steps,S,r,sigma,K,typeCP):

    typeCP = 1 if typeCP == "call" else -1
    dt = np.float(T)/steps
    SPath = np.zeros((steps+1, N)) # matrix of N paths
    SPath[0] = S # first value is S for all paths
    m = np.exp((r-0.5*sigma**2)*dt)
    for t in range(1, steps+1):
        z = npr.randn(N) # random noise for all paths at time t
        # SPath[t]=SPath[t-1]*np.exp((r-0.5*sigma**2)*dt + sigma*np.sqrt(dt)*z)
        SPath[t] = SPath[t-1]*m*np.exp(sigma*np.sqrt(dt)*z)
    # calculate payoff at maturity
    option = np.maximum(typeCP*(SPath.mean(axis=0)-K), 0)
    option = np.exp(-r*T)*option.mean()
    return option

In [75]:
%time montecarlo_dynamic(2, 10**4, 1000, 100, 0.05, 0.10, 100, "put")

Wall time: 839 ms


1.253980264129479

## Least-Squares Monte Carlo (for American options)

In [76]:
def LSM_montecarlo(T,N,steps,S,r,sigma,K, typeCP):

    typeCP = 1 if typeCP == "call" else -1    
    dt = np.float(T) / steps
    df = np.exp(-r * dt)
    
    # simulation of index levels
    SPath = np.zeros((steps + 1, N))
    SPath[0] = S
    z = npr.randn(steps, N)
    for t in range(1, steps + 1):
        SPath[t] = SPath[t-1]*np.exp((r - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*z[t-1])

    # case-based calculation of payoff
    h = np.maximum(typeCP*(SPath - K), 0)
    # LSM algorithm
    V = np.copy(h)
    for t in range(steps-1, 0, -1):
        reg = np.polyfit(SPath[t], V[t+1]*df, 7)
        C = np.polyval(reg, SPath[t])
        V[t] = np.where(C > h[t], V[t+1]*df, h[t])
    # MCS estimator
    option = df * 1 / N * np.sum(V[1])
    return option

## Old exam applications

**(2015)** A *lookback* call option pays the difference between the spot price at expiration and the minimum spot price over the term of the contract, if this difference is larger than zero. At maturity, the value of the option is:

\begin{equation}
\textit{Lookback}_T=\max \left(0, S_T-\min_{0\leq t \leq T} S_t\right).
\end{equation}

Write a Python function to value the *lookback* call option using a **binomial tree**. 

1. The function takes as inputs: the stock price at $t=0$, i.e., $S_0$, the stock volatility $\sigma$, the number of steps in the tree $N$, the risk-free rate $r$ (yearly, continuously compounded) and the maturity $T$ (in years). 
2. Use the Cox-Ross-Rubinstein model to compute the up and down steps, i.e., $u=\exp\left(\sigma \sqrt{\frac{T}{N}}\right)$ and $ud=1$. 
3. The output of the function is the price of the \textit{lookback} option. 
4. Numerical example: S0=50, $\sigma$=0.25, N=10, r=0.02, T=1.
5. Can you value this option via Monte Carlo simulation of the final price $S_T$? Discuss. 

In [33]:
def lookback(T,N,S0,r,sigma):

    ### Preliminary steps
    dt=np.float(T)/N # step size
    u=np.exp(sigma*np.sqrt(dt)) # up and down steps
    d=1/u
    p=(np.exp(r*dt)-d)/(u-d) # risk-neutral probability
    
    # Path generator (2**N paths, N steps)
    Paths=np.zeros([N+1,2**N]) # initialize paths
    Paths[0,:]=S0 # first element on path is S0

    for j in range(N,0,-1): # fill in all (2^N) possible paths in a N x 2^N matrix
        temp=2**(N-j)*([1 for i in range(0,2**(j-1))],[0 for i in range(0,2**(j-1))])
        print(temp, "\n")
        Paths[N-j+1,:]=[val for s in temp for val in s]
        print(Paths, "\n")
    
    print(Paths[1:])
    # risk-neutral probabilities
    RN_Probs=p**Paths[1:].sum(axis=0)*(1-p)**(N-Paths[1:].sum(axis=0))

    # fill the paths matrix with up and down steps
    Paths[1:]=Paths[1:]*(u-d)+d

    for i in range(1,np.size(Paths,axis=0)):
        Paths[i,:]=Paths[i-1,:]*Paths[i,:]

    ### Option price
    option_path_values=Paths[-1,:]-Paths.min(axis=0) # option value for each path

    option=np.dot(option_path_values,RN_Probs) # scalar product of value and path probability

    return option_path_values, option

In [34]:
lookback(1,2,100,0.01,0.25)

([1, 1], [0, 0]) 

[[100. 100. 100. 100.]
 [  1.   1.   0.   0.]
 [  0.   0.   0.   0.]] 

([1], [0], [1], [0]) 

[[100. 100. 100. 100.]
 [  1.   1.   0.   0.]
 [  1.   0.   1.   0.]] 

[[1. 1. 0. 0.]
 [1. 0. 1. 0.]]


(array([4.24119019e+01, 1.42108547e-14, 1.62033114e+01, 0.00000000e+00]),
 13.406035970903066)

**(2016)** A *power* call option pays a (positive) power of the difference between the spot price at expiration and a strike price, if this difference is larger than zero. At maturity, the value of the option is:
\begin{equation}
\textit{Power}_{T,k}=\max \left(0, \left[S_T-K\right]^{k}\right).
\end{equation}

1. Write a Python function to value the power call option using a binomial tree. 
2. The function takes as inputs: the stock price at $t=0$, i.e., $S_0$, the stock volatility $\sigma$, the number of steps in the tree $N$, the strike price $K$, the power coefficient $k$, the risk-free rate $r$ (yearly, continuously compounded) and the maturity $T$ (in years). 
3. Use the Cox-Ross-Rubinstein model to compute the up and down steps, i.e., $u=\exp\left(\sigma \sqrt{\frac{T}{N}}\right)$ and $ud=1$. 
4. The output of the function is the price of the lookback option. 
5. Numerical example: S0=50, sigma=0.25, N=10, r=0.02, T=1, K=50, k=2
6. How would you evaluate a plain vanilla call option using the function you just created? 
7. Can you evaluate the power option using Monte Carlo simulations? Explain.

In [77]:
def power(T,N,S0,K,k,r,sigma):
    
    ### Preliminary steps
    dt=np.float(T)/N # step size
    u=np.exp(sigma*np.sqrt(dt)) # up and down steps
    d=1/u
    p=(np.exp(r*dt)-d)/(u-d) # risk-neutral probability

    #### Path generator
    Paths=np.zeros([N+1,2**N]) # initialize paths
    Paths[0,:]=S0 # first element on path is S0

    for j in range(N,0,-1): # fill in all (2^N) possible paths in a N x 2^N matrix
        temp=2**(N-j)*([1 for i in range(0,2**(j-1))],[0 for i in range(0,2**(j-1))])
        Paths[N-j+1,:]=[val for s in temp for val in s]

    # risk-neutral probabilities
    RN_Probs=p**Paths[1:].sum(axis=0)*(1-p)**(N-Paths[1:].sum(axis=0))

    # fill the paths matrix with up and down steps
    Paths[1:]=Paths[1:]*(u-d)+d

    for i in range(1,np.size(Paths,axis=0)):
        Paths[i,:]=Paths[i-1,:]*Paths[i,:]

    ### Option price
    option_path_values=np.maximum(0,(Paths[-1,:]-K)**k) # option value for each path

    option=np.exp(-r*T)*np.dot(option_path_values,RN_Probs) # scalar product of value and path probability

    return option_path_values, Paths, option

In [78]:
def power2(T,N,S,K,k,r,sigma):

    dt=np.float(T)/N # step size
    u=np.exp(sigma*np.sqrt(dt)) # up and down steps
    d=1/u
    p=(np.exp(r*dt)-d)/(u-d) # risk-neutral probabilities

    ST=np.zeros(N+1) # Final stock prices vector
    option=np.zeros(N+1) # option price vector

    for i in range(0,N+1):
        ST[i]=S*u**(N-i)*d**i # Fill in terminal stock prices
        option[i]=max((ST[i]-K)**k,0) # Fill in terminal option prices
    
    for i in range(N-1,-1,-1):
        for j in range(0,i+1):
            option[j]=np.exp(-r*dt)*(p*option[j]+(1-p)*option[j+1])

    return option[0]

In [79]:
power(1,5,100,150,3,0.10,0.25)[2], power2(1,5,100,150,3,0.10,0.25) # two ways to solve the same problem

(784.3374468302424, 784.337446830242)